# Installation

In [1]:
!pip install streamlit opencv-python scipy pydub langchain_groq gtts TTS pyngrok ffmpeg-python SpeechRecognition gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 104.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 59.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (set

# streamlit Interface

In [4]:
%%writefile app.py
import os
import io
import base64
import tempfile
import concurrent.futures
import streamlit as st
import speech_recognition as sr
from gtts import gTTS
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from TTS.api import TTS
import soundfile as sf

# -----------------------------------------------------------------------------------------
#----------Voice Cloning Setup
#--------------Write the reference audio to a temporary file.
#------------------Generate TTS output with the cloned voice.
# -----------------------------------------------------------------------------------------

@st.cache_resource
def load_voice_cloning_model():
    """Load the voice cloning model once and cache it."""
    return TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=True)

@st.cache_resource
def load_reference_voice():
    """Load the reference voice audio from a file in the environment."""
    reference_voice_path = "/content/tom cruise.mp3"
    with open(reference_voice_path, "rb") as f:
        return f.read()

def generate_cloned_audio(text, audio_bytes, tts_model):
    """Generate audio using the voice cloning model and in-memory reference audio."""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as ref_fp:
        audio_data = io.BytesIO(audio_bytes)
        data, samplerate = sf.read(audio_data)
        sf.write(ref_fp.name, data, samplerate)

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as out_fp:
        tts_model.tts_to_file(
            text=text,
            speaker_wav=ref_fp.name,
            language="en",
            file_path=out_fp.name
        )
        generated_audio = open(out_fp.name, "rb").read()
    os.unlink(ref_fp.name)
    os.unlink(out_fp.name)
    return generated_audio

# -----------------------------------------------------------------------------------------
# Helper Functions
# -----------------------------------------------------------------------------------------
def transcribe_audio(audio_bytes):
    """Transcribe audio using SpeechRecognition."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(io.BytesIO(audio_bytes)) as source:
        audio_data_sr = recognizer.record(source)
    return recognizer.recognize_google(audio_data_sr)

def process_llm_query(recognized_text, api_key):
    """Process the LLM query using ChatGroq."""
    prompt_template = [
        ('system', 'You are a helpful assistant, your name is BlueAssistant, developed by Ayush Kumar'),
        ('user', '{input}')
    ]
    prompt = ChatPromptTemplate.from_messages(prompt_template)
    llm = ChatGroq(api_key=api_key)
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({'input': recognized_text})

def generate_tts(response_text, reference_audio=None):
    """
    Generate TTS audio using voice cloning if a reference voice is provided,
    otherwise fallback to a default gTTS voice.
    """
    try:
        if reference_audio:
            tts_model = load_voice_cloning_model()
            return generate_cloned_audio(response_text, reference_audio, tts_model)
        else:
            tts = gTTS(text=response_text, lang='en')
            mp3_fp = io.BytesIO()
            tts.write_to_fp(mp3_fp)
            return mp3_fp.getvalue()
    except Exception as e:
        st.error(f"Error in TTS generation: {e}")
        return None

# ---------------------------------------------------------------------------------------------------------
#------Main Streamlit App
#---------Load the reference voice.
#--------------- Record and Process Query Voice
#--------------------Use the recorded audio solely for transcription (query text extraction)
#------------------------Transcribe the query
#------------------------------Process query if wake word detected and if it’s a new query
#--------------------------------- Process query if wake word detected and if it’s a new query
#----------------------------------------Convert audio to base64 string for embedding in HTML
# -----------------------------------------------------------------------------------------------------------
st.title("AI Assistant with Preloaded Voice Cloning")
st.write("This app uses a preloaded reference voice for cloning responses. Record your voice query (say 'assistant' to trigger processing).")


if "reference_audio" not in st.session_state:
    st.session_state.reference_audio = load_reference_voice()

if "recognized_text" not in st.session_state:
    st.session_state.recognized_text = ""
if "last_query" not in st.session_state:
    st.session_state.last_query = ""
if "response_text" not in st.session_state:
    st.session_state.response_text = ""
if "cloned_audio" not in st.session_state:
    st.session_state.cloned_audio = None

st.header("Record Your Query")
audio_file = st.audio_input("Record your voice message (say 'assistant' to trigger processing)")
if audio_file is not None:
    audio_bytes = audio_file.read()
    st.audio(audio_bytes, format="audio/wav")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_transcribe = executor.submit(transcribe_audio, audio_bytes)
        try:
            recognized_text = future_transcribe.result(timeout=20)
            st.write("**You said:**", recognized_text)
            st.session_state.recognized_text = recognized_text
        except Exception as e:
            st.error(f"Error transcribing audio: {e}")

    if ("assistant" in recognized_text.lower() and recognized_text != st.session_state.last_query):
        st.session_state.last_query = recognized_text
        st.write("🔔 Processing query...")

        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_llm = executor.submit(
                process_llm_query,
                recognized_text,
                "your_groq_api_key"
            )
            try:
                response_text = future_llm.result(timeout=60)
                st.session_state.response_text = response_text
                st.write("**Response:**", response_text)

                with st.spinner("Generating cloned voice response..."):
                    cloned_audio = generate_tts(response_text, st.session_state.reference_audio)
                    if cloned_audio:
                        st.session_state.cloned_audio = cloned_audio
                        b64_audio = base64.b64encode(cloned_audio).decode("utf-8")
                        audio_html = f'<audio autoplay style="display:none;" src="data:audio/wav;base64,{b64_audio}"></audio>'
                        st.markdown(audio_html, unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Error processing LLM query: {e}")
else:
    st.info("Awaiting voice input.")



Overwriting app.py


# Hosting / Tunneling

In [5]:
from pyngrok import ngrok
import time

ngrok.set_auth_token("your_api_key")
public_url = ngrok.connect(8501).public_url
print("Access the UI at:", public_url)

!streamlit run app.py &
time.sleep(5)

Access the UI at: https://be45-35-198-203-107.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.198.203.107:8501



2025-03-14 11:17:27.138 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 345, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
                      ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 210, in <lambda>
    lambda m: list(m.__path__._path),
                   ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_classes.py", line 13, in __getattr__
    proxy = torch._C._get_custom_class_python_wrapper(self.name, attr)
            ^^^^^